In [27]:
from importlib import reload
from src import parsing
from src import model_path
from src import models_path
from src import model_goal
from src import models_goal
from src import model_interface
from src import visualization
from src import util
from src import config
reload(parsing)
reload(model_path)
reload(models_path)
reload(model_goal)
reload(models_goal)
reload(model_interface)
reload(visualization)
reload(util)
reload(config)

<module 'src.config' from '/home/deus/Documents/trajectory-prediction/src/config.py'>

In [16]:
# load input
# since we use a different file then for training we can use everything for eval
file_path = "data/medium_ab.csv"
_, eval_data, _ = parsing.parse_atc_day(file_path, train_ratio=0.02, eval_ratio=0.98)
input_path, correct_goals, ground_truth_path = eval_data

# limit amount of data used to avoid memory overflow
LIMIT = 5000
input_path = input_path[:LIMIT,:,:]
correct_goals = correct_goals[:LIMIT,:]
ground_truth_path = ground_truth_path[:LIMIT,:,:]
eval_data = input_path, correct_goals, ground_truth_path

100%|██████████| 1058/1058 [00:16<00:00, 62.24it/s] 


Dividing 580 human trajectories: 11 training, 569 eval, 0 test
train (2310, 40, 2) float64 (2310, 25) float32 (2310, 40, 2) float64
eval (245345, 40, 2) float64 (245345, 25) float32 (245345, 40, 2) float64


In [28]:
# generate estimates
m = model_goal.ModelGoal()
m.load('goal_simple_cnn')
goals_estimation = m.estimate(eval_data)

# free memory
m = None

In [30]:
reload(model_path)

# load predictive model
m = model_path.ModelPath(uses_goal=True)
m.load('path_simple_lstm_goal')

In [21]:
# evaluate gt goals
m.metrics(input_path, ground_truth=ground_truth_path, goals=correct_goals)

before: (5000, 40, 2)
(5000, 40, 27)
MED: 0.042653963304803234
FDE: 0.0711275687600353


In [31]:
# evaluate estimated goals
m.metrics(input_path, ground_truth=ground_truth_path, goals=goals_estimation)

MED: 0.054702254029737434
FDE: 0.10544148540413023
